In [9]:
from radd import devtests, build
data=redf_store['redata']
inits=redf_store['rebsl_boot_popt']

In [10]:
m = build.Model(data=data, inits=inits, depends_on={'v':'Cond'}, fit='subjects')
m.prepare_fit()


preparing reactive model using subjects method


In [11]:
y = m.observed.query('Cond=="bsl"').mean().iloc[1:].values

In [25]:
track='predictions' 
depends=['xx']

In [16]:
model='radd', ntrials=5000, maxfun=50, ftol=1.e-3, xtol=1.e-3, all_params=1, disp=True

{'a': 0.44409409212000012,
 'ssv': 0.94733930094933305,
 'tr': 0.30487069828933316,
 'v': 1.1223597714793327,
 'z': 0.15424655465199996}

In [77]:
y, wls, ols = devtests.compare_ols_wls_predictions(m.data, m.inits, maxfun=20)

preparing reactive model using subjects method


In [87]:
yy = [np.mean(r[6:11]) for r in [np.array(wls).mean(axis=0), np.array(ols).mean(axis=0), y]]


In [138]:
scerr = m.observed.sem().loc[200:400]*1.96
rterr = m.observed.sem().loc['c5':'c95']*1.96

In [173]:
from radd import vis
vis.scurves([np.array(wls).mean(axis=0)[1:6], y[1:6]], labels=['wls', 'data',], yerr=[scerr, scerr], linestyles=['--','-'], colors=[bpal(3)[2], rpal(3)[2]])
plt.savefig('/Users/kyle/Dropbox/SC.png', dpi=300)

In [136]:
np.vstack([w[6:11] for w in np.array(wls)]).mean(axis=1)

array([ 5.59370698,  5.64770698,  5.59770698,  5.67442698,  5.62570698,
        5.62042026,  5.55940035,  5.62838132,  5.6485091 ,  5.62768977,
        5.65499521,  5.60653108,  5.62852283,  5.62491341,  5.60046424,
        5.59107057,  5.59806265,  5.61447882,  5.62322184,  5.62103072,
        5.61249742,  5.61711928,  5.61552977])

In [140]:
yy = [np.mean(r[6:11]) for r in [np.vstack([w[6:11] for w in np.array(wls)]).mean(axis=1), y]]

In [147]:
sns.pointplot(x=np.array(['WLS','Data']), y=np.array(yy), palette='Blues')

In [157]:
[np.mean(rterr)]*2

[0.038487903145539334, 0.038487903145539334]

In [172]:
sns.set_context('notebook', font_scale=1.6)
f, ax = plt.subplots(1, figsize=(5,5))
yy = [np.mean(r[6:11])*.1 for r in [np.vstack([w[6:11] for w in np.array(wls)]).mean(axis=1), y]]

ax.errorbar(x=[1,2], y=np.array(yy), yerr=[np.mean(rterr)*.1]*2, marker='o', markersize=5, ecolor='k')

ax.set_xticks([1,2])
ax.set_xlim(.8, 2.2)
ax.set_xticklabels(np.array(['WLS','Data']))
ax.set_ylim(.52,.58)


plt.savefig('/Users/kyle/Dropbox/RT.png', dpi=300)

In [144]:
yy = [np.mean(r[6:11]) for r in [np.vstack([w[6:11] for w in np.array(wls)]).mean(axis=1), y]]

sns.pointplot(x=np.array(['WLS','Data']), y=np.array(yy), palette='Blues')

plt.setp(plt.gca(), ylim=(5.4,5.80), ylabel='RT(s)', yticks=(5.4,5.80), yticklabels=(5.4,5.80))
#plt.savefig('/Users/kyle/Dropbox/RT.png', dpi=300)

 5.4,
 5.8,

In [94]:
def plot_fits(self, bw=.1, plot_acc=False):


            sns.set_context('notebook', font_scale=1.6)

            gq = vec.loc[:, 6:11].mean()
            eq = self.observed.loc[:, 6:'11'].mean()
            fit_gq = self.fits.loc[:, 'c5':'c95'].mean()
            fit_eq = self.fits.loc[:, 'e5':'e95'].mean()

            if plot_acc:

                  f, (ax1, ax2) = plt.subplots(1,2,figsize=(10,5))

                  gacc = self.observed['Go'].mean()
                  sacc = self.observed.loc[:, 200:400].mean()
                  fit_gacc = self.fits['Go'].mean()
                  fit_sacc = self.fits.loc[:, 200:400].mean()
            else:
                  f, ax1 = plt.subplots(1, figsize=(5,5))

            # Fit RT quantiles to KDE function in radd.utils
            quant_list = [gq, fit_gq, eq, fit_eq]
            kdefits = [utils.kde_fit_quantiles(q, bw=bw) for q in quant_list]

            sns.kdeplot(kdefits[0], cumulative=True, label='data gQP', linestyle='-', color=gpal(2)[0], ax=ax1, linewidth=3.5)
            sns.kdeplot(kdefits[1], cumulative=True, label='model gQP', linestyle='--', color=gpal(2)[1], ax=ax1, linewidth=3.5)
            sns.kdeplot(kdefits[2], cumulative=True, label='data eQP', linestyle='-', color=rpal(2)[0], ax=ax1, linewidth=3.5)
            sns.kdeplot(kdefits[3], cumulative=True, label='model eQP', linestyle='--', color=rpal(2)[1], ax=ax1, linewidth=3.5)

            ax1.set_xlim(4.3, 6.5)
            ax1.set_ylabel('P(RT<t)')
            ax1.set_xlabel('RT (s)')
            ax1.set_ylim(-.05, 1.05)
            ax1.set_xticklabels(ax1.get_xticks()*.1)

            if plot_acc:
                  # Plot observed and predicted stop curves
                  vis.scurves([sacc, fit_sacc], labels=['data Stop', 'model Stop'], colors=bpal(2), linestyles=['-','--'], ax=ax2)

            plt.tight_layout()
            sns.despine()
